In [ ]:
# Debug a Processed Collection

In [2]:
%load_ext autoreload
%autoreload 2

from scripts.debug import *
from scripts.processing import *

import os

### `parameters`

In [3]:
stac_endpoint = "https://stac2.dewberryanalytics.com"  # STAC API endpoint
stac_collection = "ble_12040103_EastForkSanJacinto"  # Collection name

root_dir = r"Z:\collections\ble_12040103_EastForkSanJacinto"
nwm_flowlines_path = r"Z:\reference_data\nwm_flowlines.parquet"

db_path = os.path.join(root_dir, "ripple.gpkg")
source_models_dir = os.path.join(root_dir, "source_models")
library_dir = os.path.join(root_dir, "library")
error_report_path = os.path.join(root_dir, "error_report.xlsx")

### Create Excel Error Report

In [ ]:
%%time
dfs = []
for process_name in ["conflate_model"]:
    poll_and_update_job_status(db_path, process_name, "models")
    _, failed_reaches, _ = get_reach_status_by_process(db_path, process_name, "models")
    df = get_failed_jobs_df(failed_reaches)
    dfs.append(df)
    write_failed_jobs_df_to_excel(df, process_name, error_report_path)

In [ ]:

%%time
dfs = []
for process_name in ["extract_submodel", "create_ras_terrain", "create_model_run_normal_depth", "run_incremental_normal_depth", "run_known_wse", "create_fim_lib"]:
    # poll_and_update_job_status(db_path, process_name)
    _, failed_reaches, _ = get_reach_status_by_process(db_path, process_name)
    df = get_failed_jobs_df(failed_reaches)
    dfs.append(df)
    write_failed_jobs_df_to_excel(df, process_name, error_report_path)

### Create Composite Rasters

In [ ]:
%%time
models_files = get_models_from_stac(stac_endpoint, stac_collection)
model_ids = list(models_files.keys())
reaches = get_reaches_by_models(db_path, model_ids)

outlet_reaches = [reach[0] for reach in reaches if reach[1] is None]

run_flows2fim(
    root_dir,
    "qc",
    library_dir,
    db_path,
    outlet_reaches
    )

### Create QC Map

In [ ]:
copy_qc_map(root_dir)